In [ ]:
# install nltk, gensim and keep all texts in the text folder (or link)
# creating line-sentences for gensim.word2vec mode with nltk
import os
import nltk
import lib

In [ ]:
# prepare the text
file_path = 'text/v4_combined.txt' # 'text/lucien_sfez.txt' 
lines = lib.line_sentences(file_path)
sentences = lib.sentences_word_tokenized(lines)

def print_lines():
    for line in lines:
        print line, word_tokenize(line)
        
def print_sentences():
    for sentence in sentences:
        print sentence

In [ ]:
# build & save the model
import gensim
import math

#model.build_vocab(sentences)
model = gensim.models.Word2Vec(sentences, window = 7, workers=8)
#model.save()
model.init_sims(replace=True)
print model
print 'model memory size, mb:', model.estimate_memory()['total']/(math.pow(1024,2))
model.save('v4_combined')

In [ ]:
# Sentiment analysis of the whole text file
# plot, dumping and print of top, worst n senteces

from textblob import TextBlob
import numpy as np
import json

raw = unicode(open(file_path).read(), errors='replace')

blob = TextBlob(raw)

sentiments = []
c = 0
for sentence in blob.sentences:
    sentiments.append([sentence.sentiment.polarity,c])
    #print c,sentence
    c += 1


sen_sorted_indexed = sorted(sentiments, key= lambda s : s[0])
sen_sorted =map(lambda s : s[0],sen_sorted_indexed)


import matplotlib.pyplot as plt
%matplotlib inline
plt.figure(figsize=[30, 10])
plt.plot(sen_sorted)
plt.show()

dump_file = open('v4_combined_after_retune_sentiment_ranking.json','w')
c = 0
top_worst_n =10
for sen in sen_sorted_indexed:
    dump_file.write(json.dumps({'sen':sen,'text':blob.sentences[int(sen[1])].string}))
    if c < top_worst_n or c >= len(sen_sorted_indexed) - top_worst_n:
        print sen
        print blob.sentences[int(sen[1])]
    c += 1
dump_file.close()

In [ ]:
from nltk import Text
import lib
cool_words = []

text = Text(lib.word_tokenize(unicode(open(file_path).read(), errors='replace')))

for word in model.vocab:  

    w_tag = lib.tag(word)
    #print word, w_tag[1]
    if w_tag[1] in ['NN','VB',"VBG",'RB','NNS','ADJ','ADV','ADP']:
        cool_words.append(word)
#         text.concordance(word)
        # #nltk.help.upenn_tagset(w_tag[1])
    
print len(cool_words), 'nice words'

#  djust do that for up to 20 words
# lib.scaled_dispersion_plot(text, cool_words)

In [ ]:
# print all (noun)words in the vocabulary and their n most similar words
for w in cool_words:
    if lib.tag(w)[1] == 'NN':
        print w #, lib.tag(w)[1], lib.tag(w)[1] == "NN"
        lib.word_similar_cosmul_p(model, w,topn=3)

In [ ]:
# rest is not interesting/ running

# print sentences[0][0]
# print map(lambda w : w.encode('ascii',errors='ignore'),sentences[0])
# grammar = nltk.CFG.fromstring(sentences[0])
# parser = nltk.ChartParser(grammar)

In [ ]:
sentence = word_tokenize('this is a great tea, which you made by your own')
tagged = nltk.pos_tag(sentence)
#print tagged
entities = nltk.chunk.ne_chunk(tagged)
print entities

In [ ]:
import nltk
from nltk.corpus import treebank
from nltk.grammar import CFG, Nonterminal

tbank_productions = set(production for sent in treebank.parsed_sents()
                        for production in sent.productions())
tbank_grammar = CFG(Nonterminal('S'), list(tbank_productions))


In [ ]:
sentence = word_tokenize('this is a great tea, which you made by your own')
parser = nltk.parse.EarleyChartParser(tbank_grammar)


In [ ]:
print parser.parse(tagged)